In [1]:
import pandas as pd

In [3]:
import flask

In [5]:
import gunicorn

In [18]:
import requests

In [19]:
import sqlite3

In [20]:
conn = sqlite3.connect('data/chinook.db')

In [21]:
data = pd.read_sql_query("SELECT (c.firstname || ' ' || c.lastname) as FullName, \
i.BillingCountry as Country, i.InvoiceDate, g.Name as Genre, it.Quantity, round(i.Total,2) as Total \
FROM customers as c \
LEFT JOIN invoices as i ON i.customerid = c.customerid \
LEFT JOIN invoice_items as it ON it.invoiceid = i.invoiceid \
LEFT JOIN tracks as t ON t.trackid = it.trackid \
LEFT JOIN genres as g ON g.genreid = t.genreid", conn, parse_dates= 'InvoiceDate')
data['Month'] = data['InvoiceDate'].dt.month_name()
monthorder = ['January','February','March','April','May','June','July','August','September','October','November','December']
data['Month'] = pd.Categorical(data['Month'],
                                         categories=monthorder,
                                         ordered=True)
data.head()

,FullName,Country,InvoiceDate,Genre,Quantity,Total,Month
0,Luís Gonçalves,Brazil,2010-03-11,Sci Fi & Fantasy,1,3.98,March
1,Luís Gonçalves,Brazil,2010-03-11,Sci Fi & Fantasy,1,3.98,March
2,Luís Gonçalves,Brazil,2010-06-13,Rock,1,3.96,June
3,Luís Gonçalves,Brazil,2010-06-13,Rock,1,3.96,June
4,Luís Gonçalves,Brazil,2010-06-13,Rock,1,3.96,June


In [22]:
genre_country = pd.pivot_table(data=data,index='Month',columns=['Country','Genre'],values='Total', aggfunc=sum)
genre_country = genre_country.unstack(level=0).dropna()
genre_country

Country         Genre               Month    
Argentina       Alternative & Punk  March        27.72
                                    September     7.92
                                    November     44.55
                Easy Listening      March        27.72
                Jazz                January       3.96
                                                 ...  
United Kingdom  Rock                June         91.08
                                    July          1.98
                                    August       27.72
                                    October      83.16
                World               October      13.86
Length: 573, dtype: float64

In [23]:
sales = pd.pivot_table(data=data,
                                   index='FullName',
                                   columns='Genre',
                                  values='Total',
                                  aggfunc=sum)
sales = sales.reset_index().melt(id_vars = 'FullName')
sales = sales[sales['value'].notna()]
month = data['Month'] == 'December'
sales = sales[month]
sales.head()

,FullName,Genre,value
16,Frank Ralston,Alternative,79.30
24,Isabelle Mercier,Alternative,67.44
92,Kathy Chase,Alternative & Punk,15.84
93,Ladislav Kovács,Alternative & Punk,9.90
94,Leonie Köhler,Alternative & Punk,17.82
